In [1]:
# Library
library(ggplot2)
library(dplyr)

# Load Data and Clustering Results
dfIssues <- readRDS("../../_SharedFolder_datagotchi_federal_2024/data/pilote/dataClean/datagotchi2025_canada_pilotClustering_20250310.rds")
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# Join both datasets to have cluster, issues and id

# Sélectionner les variables issues et politiques de dfIssues
dfIssues_selected <- dfIssues %>%
  select(id, starts_with("issue_"), dv_voteChoice, dv_attitudeLeftvsRight)

# Sélectionner les variables clusters de dfUsedForClustering
dfClusters_selected <- dfUsedForClustering %>%
  select(id, cluster_name, starts_with("cluster_"))

# Fusionner les deux jeux de données sur id
dfValidation <- dfClusters_selected %>%
  inner_join(dfIssues_selected, by = "id")

# Voir les variables de ce jeu fusionné
names(dfValidation)

[1] "id"                             "cluster_name"                  
 [3] "cluster_2"                      "cluster_4"                     
 [5] "cluster_3"                      "cluster_5"                     
 [7] "cluster_1"                      "cluster_6"                     
 [9] "issue_proReconcilAboriginals"   "issue_proImmigration"          
[11] "issue_proBorderControl"         "issue_proGunControl"           
[13] "issue_reduceDeficit"            "issue_proPrivateHealth"        
[15] "issue_proUnions"                "issue_proQcIndependance"       
[17] "issue_protectFrench"            "issue_proSexEduc"              
[19] "issue_proCarbonTax"             "issue_supportOil"              
[21] "issue_climateChangeExaggerated" "issue_proLgbtqRights"          
[23] "issue_proIsrael"                "issue_helpUkraine"             
[25] "issue_mostImportant"            "issue_goodEcon"                
[27] "issue_proDistinctUSA"           "issue_anthemBilingual"         
[29] "issue_concernImmigIllegal"      "issue_immigNoHouse"            
[31] "issue_gouvActHouse"             "issue_bilingualCan"            
[33] "issue_welfareNotDeserve"        "issue_gouvNoMoreTax"           
[35] "dv_voteChoice"                  "dv_attitudeLeftvsRight"

In [3]:
generate_graph <- function(data, issue_variable, output_path) {
  
  # Convertir la variable en symbole pour dplyr et ggplot
  issue_sym <- sym(issue_variable)  # Changement ici

  # Calculer proportions et créer le graphique
  data %>%
    group_by(cluster_name, !!issue_sym) %>%
    summarise(nombre_de_repondants = n(), .groups = "drop") %>%
    group_by(cluster_name) %>%
    mutate(
      nombre_total_de_repondants = sum(nombre_de_repondants),
      proportion = nombre_de_repondants / nombre_total_de_repondants
    ) %>%
    ggplot(aes(x = !!issue_sym, y = proportion, fill = factor(cluster_name))) +
    geom_col() +
    facet_wrap(~ cluster_name) +
    labs(
      x = issue_variable,  # Correction pour utiliser le nom en string
      y = "Proportion",
      fill = "Cluster",
      title = paste("Distribution of", issue_variable, "by Cluster")
    ) +
    theme_minimal() -> graph
  
  # Sauvegarder le graphique
  ggsave(
    filename = paste0(output_path, "/validationCluster_", issue_variable, ".pdf"),
    plot = graph,
    width = 16,
    height = 12
  )
}


In [4]:
# Lister toutes les variables d'enjeux

issues_list <- colnames(dfValidation)[grepl("^issue_", colnames(dfValidation))]
issues_list <- c(issues_list, "dv_voteChoice", "dv_attitudeLeftvsRight")


In [6]:
# Lancer une boucle pour générer tous les graphiques

# Spécifier le dossier où enregistrer les graphiques
output_path <- "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025"

# Générer un graphique pour chaque enjeu sélectionné
lapply(issues_list, function(issue) {
  generate_graph(dfValidation, issue, output_path)
})


Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“for 'Baisse de l’impôt personnel' in 'mbcsToSbcs': ' substituted for ’ (U+2019)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“for 'Contrer l’inflation' in 'mbcsToSbcs': ' substituted for ’ (U+2019)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“for 'Crise de l’habitation' in 'mbcsToSbcs': ' substituted for ’ (U+2019)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“for 'Don’t care' in 'mbcsToSbcs': ' substituted for ’ (U+2019)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“for 'Don’t know' in 'mbcsToSbcs': ' substituted for ’ (U+2019)”
Warning message in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, :
“for 'L’ indépendance du Québec' in 'mbcsToSbcs': ' substituted for ’ (U+2019)”
Warning message in grid.Call.graphics(C_text, as

[[1]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proReconcilAboriginals.pdf"

[[2]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proImmigration.pdf"

[[3]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proBorderControl.pdf"

[[4]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proGunControl.pdf"

[[5]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_reduceDeficit.pdf"

[[6]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proPrivateHealth.pdf"

[[7]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proUnions.pdf"

[[8]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proQcIndependance.pdf"

[[9]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_protectFrench.pdf"

[[10]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proSexEduc.pdf"

[[11]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proCarbonTax.pdf"

[[12]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_supportOil.pdf"

[[13]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_climateChangeExaggerated.pdf"

[[14]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proLgbtqRights.pdf"

[[15]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proIsrael.pdf"

[[16]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_helpUkraine.pdf"

[[17]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_mostImportant.pdf"

[[18]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_goodEcon.pdf"

[[19]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_proDistinctUSA.pdf"

[[20]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_anthemBilingual.pdf"

[[21]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_concernImmigIllegal.pdf"

[[22]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_immigNoHouse.pdf"

[[23]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_gouvActHouse.pdf"

[[24]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_bilingualCan.pdf"

[[25]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_welfareNotDeserve.pdf"

[[26]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_issue_gouvNoMoreTax.pdf"

[[27]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_dv_voteChoice.pdf"

[[28]]
[1] "../../_PrivateFolder_datagotchi_federal_2025/graph/clustering/can_2025/validationCluster_dv_attitudeLeftvsRight.pdf"